# Regresión logística aplicada a openml

In [1]:
!pip install openml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.8 MB/s eta 0:00:00
  Created wheel for openml: filename=openml-0.14.1-py3-none-any.whl size=146923 sha256=639a375fba22ed6136cef67b281a2b2174063ffe0760bd96ca4cc0a8bb2edf6f
  Stored in directory: /root/.cache/pip/wheels/75/bc/fd/739778254a2881ef96b139d0aaf60c6d4f9130bb1459b48f10
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=0fa96382cbdacb2f573f817a9168da7acb3e8745153f2de3b90bc3987feb4721
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built openml liac-arff


In [ ]:
import warnings; warnings.filterwarnings("ignore"); import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
def get_data(data_id):
    X, y = fetch_openml(data_id=data_id, return_X_y=True, as_frame=False)
    mask = ~np.isnan(X).any(axis=1); X = X[mask, :]; y = y[mask]
    if X.shape[0] < 10: return(1.0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
    return X_train, X_test, y_train, y_test

**Ejercicio para hacer en clase: aplicar RL a un dataset de OpenML**
---

Completa el código python de la caja de abajo para para mostrar los resultados de error de train (err_train) y error de test (err_test) para combinaciones de diferentes valores de:

1.   solver (utilizar sag, saga, lbfgs y netwon-cg que son los solvers para problemas multiclase
2.   max_iter (probar valores 20, 50, 100, 200, 500)
3. C (regularización: 0.01, 0.1, 1.0 , 10.0, 100)
4. tol (tolerancia: probar valores 10e-4, 10e-2, 1.0, 10e2, 10e4)

El error de train se calcula como  err_train = 1 - accuracy_score(y_train, clf.predict(X_train))

Utliza la cabecera:

print('#   Solver     Iter       C          tol       Etr      Ete');

y la siguiente instrucción para mostrar los resultados:

print('%10s %7d %9.3f %12.4f %7.0f%% %7.0f%%' %(solver, max_iter, C, tol, err_train\*100, err_test\*100))



**IMPORTANTE: en la instrucción 2 tienes que escoger un dataset dentro del suite. Para ello consulta el cuaderno openml_all_suites_datasets.ipynb**
---


El dataset que he elegido es ha sido el Pima Indians Diabetes Database donde se empleo el algoritmo ADAP para prever el inicio de la diabetes.
El conjunto de datos esta recopilado de una población cercana a Phoenix y en específico las mujeres de herencia india pima mayores a 21 años.
y de las 768 instancias cuentan con 8 atributos cada una.

In [ ]:
import openml
#benchmark_suite = openml.study.get_suite(suite_id=99)  #escribe aquí el id del suite con el que quieres trabajar: CCC18 (99), Tabular (334), AutoML(271)
X_train, X_test, y_train, y_test = get_data(37)   #escribe aquí el id del dataste con el que quieres trabajar


In [ ]:
min_test_error = float('inf')
best_config = None
solver_errors = {}
print('#   Solver     Iter      C         tol          Etr       Ete')
for solver in ['sag','saga','lbfgs','newton-cg']:
  solver_errors[solver] = []
  for max_iter in (20,50,100,200,500):
    for C in (0.01,0.1,1.0,10.0,100):
      for tol in (10e-4,10e-2,1.0,10e2,10e4):
        clf = LogisticRegression(C=C,solver=solver,tol=tol, random_state=23, max_iter=max_iter).fit(X_train, y_train)
        err_test = 1 - accuracy_score(y_test, clf.predict(X_test))
        err_train = 1 - accuracy_score(y_train, clf.predict(X_train))
        print('%10s %7d %9.3f %12.4f %7.0f%% %7.0f%%' %(solver, max_iter, C, tol, err_train*100, err_test*100))
        solver_errors[solver].append(err_test)
        if err_test < min_test_error:
                    min_test_error = err_test
                    best_config = {'solver': solver, 'max_iter': max_iter, 'C': C, 'tol': tol}

  print('#------- ------ ---- ---- ---- ----')
print(f"Mejor Configuración: {best_config}")
print(f"Mínimo Error de Test: {min_test_error * 100:.2f}%")
for solver, errors in solver_errors.items():
    mean_error = sum(errors) / len(errors)
    print(f"Media de Error de Test para {solver}: {mean_error * 100:.2f}%")

#   Solver     Iter      C         tol          Etr       Ete
       sag      20     0.010       0.0010      31%      32%
       sag      20     0.010       0.1000      31%      34%
       sag      20     0.010       1.0000      35%      36%
       sag      20     0.010    1000.0000      35%      36%
       sag      20     0.010  100000.0000      35%      36%
       sag      20     0.100       0.0010      31%      32%
       sag      20     0.100       0.1000      31%      34%
       sag      20     0.100       1.0000      35%      36%
       sag      20     0.100    1000.0000      35%      36%
       sag      20     0.100  100000.0000      35%      36%
       sag      20     1.000       0.0010      31%      32%
       sag      20     1.000       0.1000      31%      34%
       sag      20     1.000       1.0000      35%      36%
       sag      20     1.000    1000.0000      35%      36%
       sag      20     1.000  100000.0000      35%      36%
       sag      20    10.000       0.0

Después de realizar un análisis exhaustivo de la base de datos, se han obtenido resultados significativos que nos guiarán hacia la mejor configuración posible.

Se destaca que los solvers que proporcionan los errores de prueba más bajos son lbfgs y newton-cg, evidenciando valores inferiores al 30%, mientras que saga y sag no logran alcanzar este umbral en ningún momento.

Al enfocarnos en el solver lbfgs, se observa que para un bajo número de iteraciones presenta un mayor error de prueba en comparación con iteraciones más elevadas. Asimismo, se nota que a medida que disminuyen los valores de C y la tolerancia, tanto los errores de entrenamiento como los de prueba tienden a reducirse.

Un patrón similar se observa en el solver newton-cg, inclusive mostrando una media de errores de prueba menor que la de lbfgs. En la búsqueda del menor error de prueba posible, se recomienda la selección de los siguientes hiperparámetros: solver = lbfgs, max_iter = 100, C = 1.0, tol = 0.001. Esta combinación específica arroja un error de prueba del 16.88% y un error de entrenamiento del 23%. Estos resultados indican que estos parámetros proporcionan un rendimiento óptimo en términos de precisión tanto en el conjunto de prueba como en el de entrenamiento.